# Is Solar the Answer?

This analysis investigates replacement of diesel and petrol with hydrogen for all transport in Australia.
What would this look like if the hydrogen was green and produced by solar generation.

References:
1. https://www.energy.gov.au/publications/australian-energy-statistics-table-o-electricity-generation-fuel-type-2019-20-and-2020
1. https://www.energy.gov.au/sites/default/files/Australian%20Energy%20Statistics%2C%20Table%20O%20Electricity%20generation%20by%20fuel%20type%202019-20%20and%202020.xlsx
1. https://en.wikipedia.org/wiki/Energy_density
1. https://www.energy.gov.au/sites/default/files/Australian%20Energy%20Statistics%202020%20Energy%20Update%20Report_0.pdf
1. https://www.nrel.gov/docs/fy19osti/73520.pdf

In [33]:
import pandas as pd
import numpy as np

## Current Electricity Generation

The current electricity generation numbers from [energy.gov.au](https://www.energy.gov.au/publications/australian-energy-statistics-table-o-electricity-generation-fuel-type-2019-20-and-2020)

In [145]:
df = pd.read_excel('data/Australian Energy Statistics, Table O Electricity generation by fuel type 2019-20 and 2020.xlsx', sheet_name='AUS', header=4)
df = df.drop(index=[0, 1, 7, 8, 9, 17], columns=df.columns[0]).reset_index(drop=True)
df = df.iloc[:12, :]
df = df.rename(columns={'Unnamed: 1': 'fuel'})
df = df.apply(pd.to_numeric, errors='ignore')
df['category'] = np.nan
df.loc[df.index[:5], 'category'] = 'non-renewable' 
df.loc[df.index[5:], 'category'] = 'renewable'
df = pd.pivot_table(df, columns=['category', 'fuel'])
df.index = df.index.str.replace(r'(\d\d).*-(\d\d)', r'\1\2', regex=True)
df_elec_gen = df
df_elec_gen.tail()

category  non-renewable                                                       \
fuel         Black coal    Brown coal   Natural gas   Oil products   Other a   
2016      114294.980000  48795.859000  50536.081000    5655.617000       NaN   
2017      118272.300000  43557.757000  50459.748000    5272.590000       NaN   
2018      121702.311000  36008.406000  53881.672000    4820.875747       NaN   
2019      119844.767916  34459.992520  52775.403406    4922.945279       NaN   
2020      111873.047897  33649.324322  55216.283981    4509.200664       NaN   

category       renewable                                          \
fuel       Bagasse, wood       Biogas   Geothermal         Hydro   
2016         2493.768000  1296.126000        0.215  15318.180000   
2017         2308.007000  1192.539000        0.502  16284.887000   
2018         2264.345000  1253.341000        0.000  16020.789000   
2019         2163.634583  1332.246636        0.000  15967.281111   
2020         1998.086728  1353.543192        0.000  15149.633333   

category                                                            
fuel       Small-scale solar PV          Wind Large-scale solar PV  
2016                6381.049000  12199.498000           457.212000  
2017                7399.259000  12596.985000           672.397000  
2018                8922.359000  15174.425000          1007.582000  
2019               11116.163336  17711.950833          3732.360282  
2020               13897.087569  20395.884546          7136.106674

In [146]:
df_elec_gen.groupby(level='category', axis='columns').sum().tail()

category,non-renewable,renewable
2016,219282.537000,38146.048000
2017,217562.395000,40454.576000
2018,216413.264747,44642.841000
2019,212003.109122,52023.636781
2020,205247.856864,59930.342043


In [168]:
total_renewable_supply = df_elec_gen.groupby(level='category', axis='columns').sum().loc['2020', 'renewable']
total_renewable_supply

59930.34204286933

## Specific Energy
A list of [specific energies for chemical reactions](https://en.wikipedia.org/wiki/Energy_density) was found on wikipedia.
When used to produce electricity in a fuel cell or to do work, it is the Gibbs free energy of reaction (ΔG) that sets the theoretical upper limit. For hydrogen, ΔG is 113 MJ/kg if water vapor is produced, and 118 MJ/kg if liquid water is produced. It is assumed that the value of 113 MJ/kg is the relevant value here.

In [155]:
df = pd.DataFrame.from_records([
    {'material': 'hydrogen', 'specific_energy_MJ_kg': 113},
    {'material': 'diesel', 'specific_energy_MJ_kg': 45.6},
    {'material': 'petrol', 'specific_energy_MJ_kg': 46.4}
])
df_specific_energy = df
df_specific_energy.head()

,material,specific_energy_MJ_kg
0,hydrogen,113.0
1,diesel,45.6
2,petrol,46.4


## Round Trip Efficiency
There are significant losses in process of using electricity to generate hydrogen through electrolysis, compressing the hydrogen for storage, then using a fuel cell to generate electricity from the hydrogen. These losses are captured in the 'round trip efficiency'. The value for [round trip efficiency for hydrogen](https://www.nrel.gov/docs/fy19osti/73520.pdf) obtained was `35%`

In [159]:
h2_round_trip_effic = 0.35

## Diesel Efficiency
To simplify the analysis we will assume that all transport is using Diesel, with an [efficiency](https://en.wikipedia.org/wiki/Diesel_engine#Efficiency) of `37%`

In [161]:
diesel_engine_efficiency = 0.37

## Diesel and Petrol Consumption by Transport

The energy consumption for transport in Australia was taken from [Australian Energy Update 2020](https://www.energy.gov.au/sites/default/files/Australian%20Energy%20Statistics%202020%20Energy%20Update%20Report_0.pdf)
The value from year 2018-19 was `1,748.4 PJ`

In [158]:
transport_energy_consumption = 1748.4 * 1e9 # convert to MJ

In [163]:
energy_for_work = transport_energy_consumption * diesel_engine_efficiency

## Hydrogen Equivalent
The amount of energy required to deliver similar useful energy for work via a hydrogen storage / fuel cell mechanism.
What ratio is this of current renewable generation capacity?

In [165]:
hydrogen_energy_demand = energy_for_work / h2_round_trip_effic

In [175]:
hydrogen_energy_demand_GWh =  hydrogen_energy_demand / 3.6e6
print(f'Electrical energy required for hydrogen: {hydrogen_energy_demand_GWh:0.2f} GWh')

Electrical energy required for hydrogen: 513419.05 GWh


In [172]:
hydrogen_energy_demand_renewables_ratio = hydrogen_energy_demand_GWh / total_renewable_supply
print(f'Hydrogen required {hydrogen_energy_demand_renewables_ratio:0.2f} times the current renewable supply')

Hydrogen required 8.57 times the current renewable supply


## Area Required for Solar Farms to Meet Demand
Bhadla Solar Park became the largest solar park in 2020 with a capacity of 2,245 MW. Spread across more than 5,700 hectares (14,000 acres), the park is located at Bhadla village in Jodhpur district in the Indian desert state of Rajasthan. (https://en.wikipedia.org/wiki/List_of_photovoltaic_power_stations)

In [185]:
bhadla_capacity = 2245
bhadla_area = 5700
MW_per_hectare = bhadla_capacity / bhadla_area
print(f'large scale solar generates {MW_per_hectare:0.2f} MW per hectare')

large scale solar generates 0.39 MW per hectare


In [188]:
hours_day = 12
MW_required = hydrogen_energy_demand_GWh / 365 / hours_day * 1e3
hectares_required = MW_required / MW_per_hectare
print(f'{hectares_required:0.2f} hectares are required ({hectares_required/bhadla_area:0.2f} Bhadlas) assuming {hours_day} hours/day operation')

297616.07 hectares are required (52.21 Bhadlas) assuming 12 hours/day operation


In [183]:
area_of_australia_hectares = 7656127 * 100
ratio_solar_australia = hectares_required / area_of_australia_hectares
print(f'Solar required is {ratio_solar_australia*100:0.2f}% of Australia mainland')

Solar required is 0.04% of Australia mainland
